In [1]:
import sqlalchemy as db
import pandas as pd
from sqlalchemy import create_engine
import os
from pathlib import Path
file_ext = '.csv'

In [2]:
engine = create_engine('sqlite:///../database/careportal-dev-hr-sdnn.db')
connection = engine.connect()
metadata = db.MetaData()
user_patient = db.Table('user_patient', metadata, 
                     db.Column('id', db.Integer()),
                     db.Column('doctor_id', db.Integer()),
                     db.Column('acc_path', db.VARCHAR()),
                     db.Column('hr_path', db.VARCHAR())
                     )
user_doctor = db.Table('user_doctor', metadata, 
                     db.Column('id', db.Integer()),
                     db.Column('email', db.VARCHAR()),
                     db.Column('password', db.VARCHAR())
                     )
metadata.create_all(engine)


In [5]:
print(site.columns.keys())

In [5]:
directory = '/Users/sabah/Documents/Projects/data/Patients/'

In [6]:
id_acc = []
id_hr = []
id_con = []

def get_ids(d):
    for root, dirs, files in os.walk(d):
        for file in files:
              if file.endswith(file_ext):
                filepath = os.path.join(root, file)
                filename= filepath.split('/')[8].split('.')[0].split('_')
                
                if(filename[0] == 'ACC'):
                    df_acc = pd.read_csv(filepath, usecols = ['Patient.ID'])
                    df_acc_days = df_acc['Patient.ID'].unique()[0]
                    id_acc.append(df_acc_days)

                if(filename[0] == 'HR'):
                    df_hr = pd.read_csv(filepath, usecols = ['Patient.ID'])
                    df_hr_days = df_hr['Patient.ID'].unique()[0]
                    id_hr.append(df_hr_days)
                
                if(filename[0] == 'Contact'):
                    df_con = pd.read_csv(filepath, usecols = ['Patient.ID'])
                    df_con_days = df_con['Patient.ID'].unique()[0]
                    id_con.append(df_con_days)

In [15]:
get_ids(directory)

In [7]:
list_acc_path = []
list_hr_path = []
list_con_path = []

def get_path(d):
    for root, dirs, files in os.walk(d):
        for file in files:
              if file.endswith(file_ext):
                filepath = os.path.join(root, file)
                filename = filepath.split('/')[8].split('.')[0].split('_')
                
                if(filename[0] == 'ACC'):
#                     df_acc = pd.read_csv(filepath, usecols = ['Patient.ID'])
                    list_acc_path.append(filepath)
    
                if(filename[0] == 'HR'):
                    list_hr_path.append(filepath)
                
                if(filename[0] == 'Contact'):
                    list_con_path.append(filepath)

In [8]:
get_path(directory)

In [6]:
list_acc_path[0].split('/')

In [9]:
hr_path = []
acc_path = []
def get_relativePath():
    for item in list_hr_path:
        head_tail = os.path.split(item)
        dir_tail = os.path.split(head_tail[0])
        rel_path = '/'+dir_tail[1] + '/' + head_tail[1]
        hr_path.append(rel_path)
        
    for item in list_acc_path:
        head_tail = os.path.split(item)
        dir_tail = os.path.split(head_tail[0])
        rel_path = '/'+dir_tail[1] + '/' + head_tail[1]
        acc_path.append(rel_path)

In [10]:
get_relativePath()

In [16]:
dict_acc = {
            'id': id_acc,
            'acc_path':acc_path
           }

In [18]:
dict_hr = {
            'id': id_hr, 
            'hr_path':hr_path
           }

In [19]:
df_acc_path = pd.DataFrame(dict_acc)
df_hr_path = pd.DataFrame(dict_hr)

In [85]:
df_acc_path

,id,acc_path
0,75,/PatientID_75/ACC_75.csv
1,81,/PatientID_81/ACC_81.csv
2,158,/PatientID_158/ACC_158.csv
3,72,/PatientID_72/ACC_72.csv
4,151,/PatientID_151/ACC_151.csv
...,...,...
109,38,/PatientID_38/ACC_38.csv
110,115,/PatientID_115/ACC_115.csv
111,36,/PatientID_36/ACC_36.csv
112,918,/PatientID_918/ACC_918.csv


In [20]:
df_hr_path

,id,hr_path
0,75,/PatientID_75/HR_75.csv
1,81,/PatientID_81/HR_81.csv
2,158,/PatientID_158/HR_158.csv
3,72,/PatientID_72/HR_72.csv
4,151,/PatientID_151/HR_151.csv
...,...,...
111,38,/PatientID_38/HR_38.csv
112,115,/PatientID_115/HR_115.csv
113,36,/PatientID_36/HR_36.csv
114,918,/PatientID_918/HR_918.csv


In [21]:
df = pd.merge(df_acc_path, df_hr_path, on='id', how='outer')

In [22]:
df

,id,acc_path,hr_path
0,75,/PatientID_75/ACC_75.csv,/PatientID_75/HR_75.csv
1,81,/PatientID_81/ACC_81.csv,/PatientID_81/HR_81.csv
2,158,/PatientID_158/ACC_158.csv,/PatientID_158/HR_158.csv
3,72,/PatientID_72/ACC_72.csv,/PatientID_72/HR_72.csv
4,151,/PatientID_151/ACC_151.csv,/PatientID_151/HR_151.csv
...,...,...,...
111,36,/PatientID_36/ACC_36.csv,/PatientID_36/HR_36.csv
112,918,/PatientID_918/ACC_918.csv,/PatientID_918/HR_918.csv
113,31,/PatientID_31/ACC_31.csv,/PatientID_31/HR_31.csv
114,139,NaN,/PatientID_139/HR_139.csv


In [24]:
df.to_csv('/Users/sabah/Desktop/path.csv', index=False)

In [123]:
df.to_sql('user_patient', con = engine, if_exists = 'replace', index = False)

In [4]:
df_edit = pd.read_csv('path.csv')

In [5]:
df_edit

,id,acc_path,hr_path
0,75,/PatientID_75/ACC_75.csv,/PatientID_75/HR_75.csv
1,81,/PatientID_81/ACC_81.csv,/PatientID_81/HR_81.csv
2,158,/PatientID_158/ACC_158.csv,/PatientID_158/HR_158.csv
3,72,/PatientID_72/ACC_72.csv,/PatientID_72/HR_72.csv
4,151,/PatientID_151/ACC_151.csv,/PatientID_151/HR_151.csv
...,...,...,...
111,36,/PatientID_36/ACC_36.csv,/PatientID_36/HR_36.csv
112,918,/PatientID_918/ACC_918.csv,/PatientID_918/HR_918.csv
113,31,/PatientID_31/ACC_31.csv,/PatientID_31/HR_31.csv
114,139,NaN,/PatientID_139/HR_139.csv


In [6]:
df_edit['doctor_id'] = 0

In [7]:
df_edit

,id,acc_path,hr_path,doctor_id
0,75,/PatientID_75/ACC_75.csv,/PatientID_75/HR_75.csv,0
1,81,/PatientID_81/ACC_81.csv,/PatientID_81/HR_81.csv,0
2,158,/PatientID_158/ACC_158.csv,/PatientID_158/HR_158.csv,0
3,72,/PatientID_72/ACC_72.csv,/PatientID_72/HR_72.csv,0
4,151,/PatientID_151/ACC_151.csv,/PatientID_151/HR_151.csv,0
...,...,...,...,...
111,36,/PatientID_36/ACC_36.csv,/PatientID_36/HR_36.csv,0
112,918,/PatientID_918/ACC_918.csv,/PatientID_918/HR_918.csv,0
113,31,/PatientID_31/ACC_31.csv,/PatientID_31/HR_31.csv,0
114,139,NaN,/PatientID_139/HR_139.csv,0


In [9]:
df_edit.to_csv('path_doc_id.csv', index=False)

In [3]:
df_edit = pd.read_csv('path_doc_id.csv')


In [4]:
df_edit.to_sql('user_patient', con = engine, if_exists = 'replace', index = False)